# Homolgy Modelling

This notebook shows examples of how to perform homology modelling in KinoML. 

In [1]:
# get relevant imports
from kinoml.modeling.homology import HomologyModel
from kinoml.core.proteins import ProteinStructure

To start, we need a template from which to base our homolgy model on.

In [2]:

hm = HomologyModel()

# If already have our model (e.g. if prepared using Spruce TK etc), so we can load it easily
structure = ProteinStructure.from_file("./4yne_protein.pdb")

# If we just want to grab a structure from the PDB we could use the `from_name` attribute in `ProteinStructure`, e.g:
# structure_pdb = ProteinStructure.from_name('4yne')

# Once we have our structure, we need to extract it's sequence. Note, this will often not be the canonical sequence.
sequence = structure.sequence

We now have our template structure and its sequence. If, for some reason, we didn't have access to the structure but did have access to the sequence we could run a BLAST search to find a PDB structure for our template:

In [3]:
model_template = hm.get_pdb_template(sequence)

@> Blast searching NCBI PDB database for "GSGIR..."
@> Blast search completed in 16.0s.


In [4]:
model_template.metadata

{'id': '4yne'}

In this toy example our BLAST search using the `4YNE` sequence has, as expected, returend the `4YNE` structure in the PDB as the "best" model for us to use as our template.

Typically, we will want to search the PDB with a query sequence and find the most relevant PDB structure to use as our template. We can use the `get_uniprot_sequence` attribute to obtain the full canonical sequence from UniProt based on the UniProt ID.

In [5]:
up_seq = hm.get_uniprot_sequence('P04629')

In [6]:
up_seq

'MLRGGRRGQLGWHSWAAGPGSLLAWLILASAGAAPCPDACCPHGSSGLRCTRDGALDSLHHLPGAENLTELYIENQQHLQHLELRDLRGLGELRNLTIVKSGLRFVAPDAFHFTPRLSRLNLSFNALESLSWKTVQGLSLQELVLSGNPLHCSCALRWLQRWEEEGLGGVPEQKLQCHGQGPLAHMPNASCGVPTLKVQVPNASVDVGDDVLLRCQVEGRGLEQAGWILTELEQSATVMKSGGLPSLGLTLANVTSDLNRKNVTCWAENDVGRAEVSVQVNVSFPASVQLHTAVEMHHWCIPFSVDGQPAPSLRWLFNGSVLNETSFIFTEFLEPAANETVRHGCLRLNQPTHVNNGNYTLLAANPFGQASASIMAAFMDNPFEFNPEDPIPVSFSPVDTNSTSGDPVEKKDETPFGVSVAVGLAVFACLFLSTLLLVLNKCGRRNKFGINRPAVLAPEDGLAMSLHFMTLGGSSLSPTEGKGSGLQGHIIENPQYFSDACVHHIKRRDIVLKWELGEGAFGKVFLAECHNLLPEQDKMLVAVKALKEASESARQDFQREAELLTMLQHQHIVRFFGVCTEGRPLLMVFEYMRHGDLNRFLRSHGPDAKLLAGGEDVAPGPLGLGQLLAVASQVAAGMVYLAGLHFVHRDLATRNCLVGQGLVVKIGDFGMSRDIYSTDYYRVGGRTMLPIRWMPPESILYRKFTTESDVWSFGVVLWEIFTYGKQPWYQLSNTEAIDCITQGRELERPRACPPEVYAIMRGCWQREPQQRHSIKDVHARLQALAQAPPVYLDVLG'

We can then run a blast search using this canonical sequence to search the PDB for the best template model to use. **NOTE: This is still in test phase**

In [ ]:
model_template2 = hm.get_pdb_template(up_seq)

In [ ]:
model_template2.metadata